In [ ]:
	
import subprocess
from pydub import AudioSegment
import math
import openai
import glob

# video -> audio
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg", 
        "-i",
        video_path,
        "-vn", # no video
        audio_path,
    ]
    subprocess.run(command) # terminal 


def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path) # load
    chunk_len = chunk_size * 60 * 1000 #milisec
    chunks = math.ceil(len(track) / chunk_len) 
    for i in range(chunks): # create chucnks round up 
        start_time = i * chunk_len # ex) 00:00 - 10:00
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(
            f"./{chunks_folder}/chunk_{i}.mp3",
            format="mp3",
        )


def transcribe_chunks(chunk_folder, destination): #chunk_folder 안에서 mp3찾고
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files: # 파일을 열어서 audio_file로부르고, 
        with open(file, "rb") as audio_file, open(destination, "a") as text_file: #"a"= append
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            text_file.write(transcript["text"])